In [1]:
import pandas as pd 
import lightgbm as lgbm 
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df = pd.read_csv('https://raw.githubusercontent.com/Matthew-Booth-Py/Datasets/main/house-price/train.csv?token=GHSAT0AAAAAABXNSTQGLR4HGN4QYUP5KKYAYZ6DZ6Q')
df = df.select_dtypes(['float','int']).drop(columns=['Id']).fillna(df.mean())

/var/folders/md/c9lgr11n7x1c0_tghtfh_kjr0000gn/T/ipykernel_6007/4136769931.py:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df = df.select_dtypes(['float','int']).drop(columns=['Id']).fillna(df.mean())


In [2]:
X = StandardScaler().fit_transform(df.drop(columns=['SalePrice']))
y = df['SalePrice']

In [9]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0)

# Baseline Model

In [4]:
m = lgbm.LGBMRegressor()
m.fit(X_train,y_train)
r2_score(y_test, m.predict(X_test))

0.8615964998893736

# PCA 

In [5]:
pca = PCA(n_components=5)
components = pca.fit_transform(X_train)
Z = pd.DataFrame(data=components)
print(pca.explained_variance_ratio_)

[0.19182215 0.0898612  0.06960899 0.05538939 0.04166799]


In [6]:
m = lgbm.LGBMRegressor()
m.fit(Z, y_train)
X_test = pca.transform(X_test)
r2_score(y_test, m.predict(X_test))

0.8231688555989081

**Sacrifice Accuracy for Explainability**

- 20% of the classification information is contained by the first principle component. 

# Recursive Feature Elimination
- As the name states, RFE recursively eliminates feautres from the feature set once they've been tested through various combinations and weighted in terms of it's predictive value with respect to the target variable. 

In [11]:
Z = df.drop(columns=['SalePrice'])
rfe = RFE(lgbm.LGBMRegressor(), )
m = rfe.fit(X_train,y_train)

In [13]:
r2_score(y_test, m.predict(X_test))

0.8363705941922115